In [2]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# Access the variables
APIFY_TOKEN = os.getenv("APIFY_TOKEN")

# 1. API parse jobs

## Live running the Apify, could simiply request the output instead

In [ ]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient(APIFY_TOKEN)

# Prepare the Actor input
run_input = {
    "queries": "Teacher\nhttps://www.google.com/search?q=doctor&ibp=htl;jobs",
    "maxPagesPerQuery": 1,
    "csvFriendlyOutput": False,
    "languageCode": "",
    "maxConcurrency": 10,
    "saveHtml": False,
    "saveHtmlToKeyValueStore": False,
    "includeUnfilteredResults": False,
}

# Run the Actor and wait for it to finish
run = client.actor("SpK8RxKhIgV6BWOz9").call(run_input=run_input)

# Fetch and print Actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)

In [5]:
def traverse_json(obj, indent=''):
    if isinstance(obj, dict):
        for key, value in obj.items():
            print(f"{indent}{key}:")
            traverse_json(value, indent + '  ')
    elif isinstance(obj, list):
        for item in obj:
            traverse_json(item, indent + '  ')
    else:
        print(f"{indent}{obj}")

# Replace 'json_object' with the actual JSON object you want to traverse

traverse_json(item)


searchQuery:
  term:
    doctor
  page:
    1
  type:
    SEARCH
  domain:
    google.com
  countryCode:
    US
  languageCode:
    None
  locationUule:
    None
  resultsPerPage:
    10
url:
  https://www.google.com/search?q=doctor&ibp=htl;jobs
hasNextPage:
  True
googleJobs:
    title:
      Veterinarian Doctor
    companyName:
      Lap of Love
    location:
        Vancouver, WA   
    via:
      via BeBee
    description:
      Are you ready for a change of pace from in-clinic practice? Looking to reconnect with pet families and provide the kind of service you'd always envisioned by having generous appointment times and minimal administrative work? At Lap of Love, our mission is to provide support to pets and their families during one of life's most challenging moments by helping pets pass peacefully in the comfort of... their own home.
Established and owned by veterinarians, we recognize that pets are more than just animals — they're beloved family members. That's why we prioriti

In [7]:
next(client.dataset(run["defaultDatasetId"]).iterate_items())

{'searchQuery': {'term': 'Teacher',
  'page': 1,
  'type': 'SEARCH',
  'domain': 'google.com',
  'countryCode': 'US',
  'languageCode': None,
  'locationUule': None,
  'resultsPerPage': 10},
 'url': 'https://www.google.com/search?ibp=htl;jobs&q=Teacher',
 'hasNextPage': True,
 'googleJobs': [{'title': 'Child Care Teacher',
   'companyName': 'Bright Horizons Family Solutions',
   'location': '  Gaithersburg, MD   ',
   'via': 'via Bright Horizons Careers',
   'description': 'Imagine your future as a teacher with a world-class team where you make a difference for children every day. Imagine learning from experts in your field, and having the opportunity to earn your college degree – for free. Imagine it all as a Bright Horizons Teacher.\n\nFull-time positions now available with Infants and Toddlers at our Bright Horizons at Discovery Meadows location in Gaithersburg... Maryland.\n\nA Bright Horizons Career Includes:\n• Flexible scheduling\n• Medical, dental, and vision insurance\n• 401(k

## Request the output by calling REST

In [2]:
import os
import requests
import json
import boto3

# # Make sure the data folder exists
# data_folder = "data"
# if not os.path.exists(data_folder):
#     os.makedirs(data_folder)

# Make the GET request
response = requests.get(
    f"https://api.apify.com/v2/datasets/mj6RWZ0pjBY2aLhmq/items?token={APIFY_TOKEN}"
)

# Check if the request was successful
if response.status_code == 200:
    # Save the response content as JSON
    data = response.json()

    FILE_NAME = "../data/output.json"
    # Save the JSON data to a file
    with open(FILE_NAME, "w") as file:
        json.dump(data, file)

    # Upload the JSON file to S3 bucket
    s3 = boto3.client("s3")  # Boto3 will automatically search for credentials
    bucket_name = "job-recommender-bucket-yusa"
    s3.upload_file(FILE_NAME, bucket_name, FILE_NAME)
else:
    print("Error: Failed to retrieve data")

## Peak the APIFY returned JSON

In [3]:
len(data)

4

In [5]:
data[0]

{'searchQuery': {'term': 'data science jobs in Canada',
  'page': 1,
  'type': 'SEARCH',
  'domain': 'google.com',
  'countryCode': 'US',
  'languageCode': None,
  'locationUule': None,
  'resultsPerPage': 10},
 'url': 'https://www.google.com/search?q=data+science+jobs+in+Canada&oq=google+jobs&gs_lcrp=EgZjaHJvbWUqCggCEAAYsQMYgAQyDggAEEUYJxg7GIAEGIoFMgYIARBFGEAyCggCEAAYsQMYgAQyCggDEAAYkgMYgAQyCggEEAAYsQMYgAQyBggFEEUYPDIGCAYQRRhBMgYIBxBFGEHSAQg0MTM3ajBqN6gCALACAA&sourceid=chrome&ie=UTF-8&ibp=htl;jobs&sa=X&ved=2ahUKEwiSkb2Ov4uDAxWRIDQIHWImCI0QudcGKAF6BAggECo&sxsrf=AM9HkKnL3kWnQdaFMTTB0VMmiGhqogChaA:1702438898722#fpstate=tldetail&htivrt=jobs&htidocid=5Au9kxSU8z87m4ltAAAAAA%3D%3D',
 'hasNextPage': True,
 'googleJobs': [{'title': 'Data Scientist',
   'companyName': 'Teranet Inc.',
   'location': '  Toronto, ON, Canada   ',
   'via': 'via LinkedIn',
   'description': 'Who We Are\n\nTeranet is Canada’s leader in the delivery and transformation of statutory registry services with extensive expe

In [7]:
page_1_jobs = data[1]["googleJobs"]

In [8]:
len(page_1_jobs)

10

In [9]:
page_1_jobs[0].keys()

dict_keys(['title', 'companyName', 'location', 'via', 'description', 'jobHighlights', 'relatedLinks', 'thumbnail', 'extras', 'metadata', 'applyLink'])

In [13]:
job_0 = page_1_jobs[0]
job_0

{'title': 'Data Scientist II (Canada Remote)',
 'companyName': 'Fullscript',
 'location': '  Ottawa, ON, Canada   ',
 'via': 'via Work In Tech Job Board - Communitech',
 'description': "Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.\n\nWe know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.\n\nFullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.\n\nWith over 5 million

In [34]:
job_0["extras"][0]

'Full-time'

In [30]:
job_0['applyLink']['link']

'https://www1.communitech.ca/companies/fullscript/jobs/31257911-data-scientist-ii-canada-remote?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic'

### Note that the description and jobHighlights have the same information

In [28]:
print(job_0['description'])

Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.

We know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.

Fullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.

With over 5 million patients served and 70,000+ practitioners supported across North America, we’re just getting started.

Come build a healthier future with us.

The Data team is a key strategic partner at... F

In [26]:
test_description = job_0["jobHighlights"][0]["items"][0]
print(test_description)

Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.

We know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.

Fullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.

With over 5 million patients served and 70,000+ practitioners supported across North America, we’re just getting started.

Come build a healthier future with us.

The Data team is a key strategic partner at... F

## Read out temp local data 

In [22]:
import json
import pprint

# Specify the file path
file_path = "../data/output.json"

# Read the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Print the data
pprint.pprint(data)


[{'categories': [{'options': [{'text': 'All'},
                              {'text': 'Data scientist',
                               'value': 'data scientist'},
                              {'text': 'Data analyst', 'value': 'data analyst'},
                              {'text': 'Manager', 'value': 'manager'},
                              {'text': 'Data science', 'value': 'data science'},
                              {'text': 'Analyst', 'value': 'analyst'},
                              {'text': 'Director', 'value': 'director'},
                              {'text': 'Senior', 'value': 'senior'},
                              {'text': 'Analytics consultant',
                               'value': 'analytics consultant'},
                              {'text': 'Assistant professor',
                               'value': 'assistant professor'},
                              {'text': 'Associate director',
                               'value': 'associate director'},
             

In [23]:
def apify_get_clean():
    jobs = run_actor()
    batch_jobs = []
    for page in jobs:
        jobs_info = page["googleJobs"]
        batch_jobs.extend(jobs_info)

    print(len(batch_jobs), "been scraped")
    return batch_jobs

In [24]:
job_0 = data[1]["googleJobs"][0]
job_0

{'title': 'Data Scientist II (Canada Remote)',
 'companyName': 'Fullscript',
 'location': '  Ottawa, ON, Canada   ',
 'via': 'via Work In Tech Job Board - Communitech',
 'description': "Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.\n\nWe know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.\n\nFullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.\n\nWith over 5 million

In [9]:
job_0.keys()

dict_keys(['title', 'companyName', 'location', 'via', 'description', 'jobHighlights', 'relatedLinks', 'thumbnail', 'extras', 'metadata', 'applyLink'])

In [12]:
def filter_full_job_info(job):
    keys_to_filter = [
        "title",
        "companyName",
        "location",
        "via",
        "description",
        "metadata",
        # "applyLink",
    ]
    job_filtered = {key: job[key] for key in keys_to_filter}
    job_filtered["applyLink"] = job["applyLink"]["link"]
    job_filtered["metadata"] = "".join(job["metadata"].values())

    return job_filtered

In [13]:
filter_full_job_info(job_0)

{'title': 'Data Scientist II (Canada Remote)',
 'companyName': 'Fullscript',
 'location': '  Ottawa, ON, Canada   ',
 'via': 'via Work In Tech Job Board - Communitech',
 'description': "Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.\n\nWe know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.\n\nFullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.\n\nWith over 5 million

In [10]:
# Serailice the dict for embedding

job_0_str = json.dumps(job_0)
print(job_0_str)


{"title": "Data Scientist II (Canada Remote)", "companyName": "Fullscript", "location": "  Ottawa, ON, Canada   ", "via": "via Work In Tech Job Board - Communitech", "description": "Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.\n\nWe know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.\n\nFullscript\u2019s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare\u2019s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can\u2019t be found anywhere else.\n\nWith ove

# 2. Langchain parsing

## a) Chains

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=3000)

In [42]:
json_parsed_sample = {
    "job_basics": {
        "title": "Machine Learning Engineer",
        "company": "Google",
        "industry": "Technology",
        "location": "Vancouver, BC",
        "years_of_experience_required": "3+ years",
        "employment_type": "Full-time",
        "source": "LinkedIn",
        "application_link": "https://...",
    },
    "required_skills": {
        "programming_languages": ["Python", "Rust", "Go", "JavaScript"],
        "data_visualization_tools": ["Tableau", "PowerBI", "Looker"],
        "databases": ["PostgreSQL", "MySQL", "MongoDB"],
        "web_development": {
            "front_end_technologies": ["HTML", "CSS", "React", "Flask"],
            "back_end_technologies": ["Node.js", "Django"],
        },
        "devops_tools": ["Git", "Docker", "Kubernetes"],
        "statistical_analysis_methods": ["Linear Regression", "Logistic Regression"],
        "machine_learning_frameworks": ["PyTorch", "TensorFlow", "Scikit-learn", "hugingface"],
        "machine_learning_operations": ["MLFlow", "Kubeflow", "Seldon", ],
        "cloud_platforms": ["AWS", "GCP", "Azure"],
        "big_data_technologies": ["Hadoop", "Spark"],
        "natural_language_processing": ["NLTK", "SpaCy"],
        "deep_learning_concepts": ["CNN", "RNN", "LSTM"],
        "data_engineering_tools": ["Airflow", "Luigi"],
        "large_models": ["GPT-3", "BERT", "Transformer-based models"],
    },
    "company_culture": ["work-life balance", "team environment", "flexible hours", "diversity"],
    "job_benefits": {
        "salary_range": "100k - 150k",
        "salary_currency": "CAD",
        "paid_time_off": "14 days",
        "retirement_plan_matching": True,
        "professional_development_budget": "$1500",
        "employee_discounts": "20% off up to $5000 yearly",
        "remote_work_policy": {
            "fully_remote_option": False,
            "partial_remote_option": "2 days/week",
            "work_from_anywhere_program": "Up to 90 days/year",
        },
        "relocation_support": False,
    },
}

        # "description": "Detailed job description goes here...",
        # "full_text_of_benefits": "Full description of all benefits offered...",


In [43]:
system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template="You are an AI assistant trained to assist job seekers by parsing job descriptions into structured JSON format. \
                  When provided with a job description, extract relevant information and categorize it under the appropriate JSON keys as shown in the example. \
                  If a field is not explicitly mentioned in the job description, use your understanding to infer the information. If it cannot be inferred, leave the field as an empty string. \
                  Please follow the structure of the example JSON object below to format the parsed information correctly. \
                  {json_parsed_sample} \ Note: The input job description may not include all details. Only fill in the fields for which information is provided or can be reasonably inferred.",
        input_variables=["json_parsed_sample"],
    )
)

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="The following is the job description \n {json}?",
        input_variables=["json"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
chat = ChatOpenAI(temperature=0.9, model="gpt-4-1106-preview")
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
parsed_jd = chain.run(json_parsed_sample=json_parsed_sample, json=job_0)

print(parsed_jd)

```json
{
  "job_basics": {
    "title": "Data Scientist II (Canada Remote)",
    "company": "Fullscript",
    "industry": "Health Technology",
    "location": "Ottawa, ON, Canada",
    "years_of_experience_required": "2+ years",
    "employment_type": "Full-time",
    "source": "Work In Tech Job Board - Communitech",
    "application_link": "https://www1.communitech.ca/companies/fullscript/jobs/31257911-data-scientist-ii-canada-remote?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic"
  },
  "required_skills": {
    "programming_languages": ["Python"],
    "data_visualization_tools": ["Business Intelligence systems"],
    "databases": ["relational databases", "Snowflake", "MySQL"],
    "web_development": {
      "front_end_technologies": ["HTML", "CSS"]
    },
    "devops_tools": [],
    "statistical_analysis_methods": [],
    "machine_learning_frameworks": [],
    "machine_learning_operations": [],
    "cloud_platforms": [],
    "big_data_technologies": [

In [40]:
pprint.pprint(parsed_jd)

("{'job_basics': {'title': 'Data Scientist II (Canada Remote)', 'company': "
 "'Fullscript', 'industry': '', 'location': 'Ottawa, ON, Canada', "
 "'years_of_experience_required': '2+ years', 'employment_type': 'Full-time', "
 "'source': 'Work In Tech Job Board - Communitech', 'application_link': "
 "'https://www1.communitech.ca/companies/fullscript/jobs/31257911-data-scientist-ii-canada-remote?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', "
 "'description': 'Embracing a new era in patient care, Fullscript is a "
 'trailblazing health technology company changing the way health is '
 'prescribed.\\n\\nWe know the conventional model of patient care is ripe for '
 'innovation. At Fullscript, our work is guided by our mission of creating '
 'meaningful change for the future of our healthcare systems, so that '
 'integrative medicine will one day simply be called '
 'medicine.\\n\\nFullscript’s care delivery platform combines customized '
 'treatment planni

## b) Load parsed job info to Dynamo DB

In [34]:
DynamoDB_TABLE_NAME = "jobs-db-yusa"

In [37]:
## Explorer other saving options
import boto3

# Create a DynamoDB client
dynamodb = boto3.client("dynamodb")

# Define the table name
table_name = DynamoDB_TABLE_NAME

# Define the dictionary to be saved
data = {
    "id": {"S": "1"},
    "job_title": {"S": "Data Scientist II (Canada Remote)"},
    "company_name": {"S": "Fullscript"},
    "location": {"S": "Ottawa, ON, Canada"},
    "source": {"S": "via Work In Tech Job Board - Communitech"},
    "full_text_description": {
        "S": "Embracing a new era in patient care... [truncated for brevity]"
    },
    "job_type": {"S": "Full-time"},
    "link": {
        "S": "https://www1.communitech.ca/companies/fullscript/jobs/31257911-data-scientist-ii-canada-remote?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic"
    },
}


# Save the dictionary to DynamoDB
response = dynamodb.put_item(TableName=table_name, Item=data)

# Print the response
print(response)

{'ResponseMetadata': {'RequestId': 'IM9MVTOSTS020M2BNFP47219TFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 05 Jan 2024 04:12:44 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'IM9MVTOSTS020M2BNFP47219TFVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '2745614147'}, 'RetryAttempts': 0}}


In [30]:
parsed_jd_temp = {
    "job_title": "Data Scientist II (Canada Remote)",
    "company_name": "Fullscript",
    "location": "Ottawa, ON, Canada",
    "source": "via Work In Tech Job Board - Communitech",
    "full_text_description": "Embracing a new era in patient care, Fullscript is a trailblazing health technology company changing the way health is prescribed.\n\nWe know the conventional model of patient care is ripe for innovation. At Fullscript, our work is guided by our mission of creating meaningful change for the future of our healthcare systems, so that integrative medicine will one day simply be called medicine.\n\nFullscript’s care delivery platform combines customized treatment planning, wellness protocols, data insights and ongoing wellness support, with the ability to prescribe and manage the use of healthcare’s best supplements all in one place. Above all else, Fullscript is trusted by practitioners due to an unrelenting focus on quality and a patient experience that can’t be found anywhere else.\n\nWith over 5 million patients served and 70,000+ practitioners supported across North America, we’re just getting started.\n\nCome build a healthier future with us.\n\nThe Data team is a key strategic partner at... Fullscript, responsible for providing analytical support for all internal functions to drive effective planning and decision making. Reporting to a Technical Lead, the Data Scientist will perform in-depth exploration and analysis of company data in order to guide goals, improve processes, and help our leaders understand and track key metrics.\nWhat you'll do\n• Partner with key stakeholders to dive deep into the business challenges and identify insights to drive company strategies\n• Build statistical models and partner with our machine learning team to productionize impactful machine learning solutions\n• Improve data availability and data literacy across the organization\n• Collaborate with the Analytics Engineering team to improve the availability of reliable data and automation of recurring business needs\n• Partner with stakeholders to create meaningful goals and targets for both team and company-level initiatives\n• Support, facilitate, and monitor strategic objectives through the creation of data science models, segmentation, and analytical testing\n• Guide and advise business functions on the collection of data to broaden our visibility into company activities\n• Proactively contribute to our shared documentation store and self-serve Business Intelligence artifacts\n• Proactively watch for opportunities to improve code quality and scalability within our codebase\n• Define, design, and develop dashboards and artifacts for key metrics and KPIs using various data languages and Business Intelligence systems\n• Other duties may be assigned as determined by the Company from time to time\nWhat you bring to the table\n• Minimum Bachelor’s Degree in a quantitative, analytical, or similar field with 2+ years of relevant experience successfully working in a data role using relational databases\n• Experience building Statistical and Machine Learning models using a programming language like Python.\n• Experience working with Business Intelligence systems and tools (BI tooling, databases, spreadsheets)\n• Proficiency in SQL including basic transformations\n• Creative problem-solving skills with a passion for uncovering strategic opportunities\n• Proven ability to work independently in a fast-paced environment, managing competing priorities on multiple simultaneous projects\n• Demonstrated attention to detail\n• Strong written and verbal communication skills with the ability to effectively communicate complex ideas and concepts to both technical and non-technical teams\nBonus if you have:\n• Experience with Snowflake, MySQL\n• Experience with Looker\n• Experience with Python\n• Experience with HTML/CSS\n• Experience in B2B or SaaS, working in a high-growth environment\n• Interest or experience in Health Tech, Integrative Medicine, or supplements\nWhat we can offer you\n• Generous PTO and competitive pay\n• Fullscript’s RRSP match program for financial health\n• Flexible benefits package and workplace wellness program\n• Training budget and company-wide learning initiatives\n• Discount on Fullscript catalog of products\n• Ability to work Wherever You Work Well*\n• Our Wherever You Work Well philosophy means Fullscript teammates get to pick their own office — whether that’s in-office, at home, or a bit of both 🐶🏡\n\nFullscript is committed to diversity in its workforce and is proud to be an equal opportunity employer. We are excited to work with talented people, period. All employment decisions are based on business needs, job requirements, and individual qualifications, without regard to race, color, religion or belief, national or ethnic origin, gender, age, disability, sexual orientation, gender identity and/or expression, marital or civil status, political affiliation, family or parental status, or any other status protected by the laws or regulations in the jurisdictions in which we operate.\n\nAccommodations are available on request for candidates taking part in all aspects of the selection process. Please send an email to accommodations@fullscript.com and let us know the nature of your request and your contact information.\n\nOur team handles a lot of sensitive information, which means we require all candidates that receive and accept employment offers to complete a background check before being hired",
    "job_type": "Full-time",
    "link": "https://www1.communitech.ca/companies/fullscript/jobs/31257911-data-scientist-ii-canada-remote?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic",
}

In [33]:
list(parsed_jd_temp.keys())

['job_title',
 'company_name',
 'location',
 'source',
 'full_text_description',
 'job_type',
 'link']

In [13]:
from openai import OpenAI
client = OpenAI()


def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


job_0_emb = get_embedding(job_0_str)

In [15]:
len(job_0_emb)

1536

In [43]:
job_0_emb

[0.002754964167252183,
 0.006326977163553238,
 0.0019464981742203236,
 -0.02489595115184784,
 -0.014130134135484695,
 0.03880637511610985,
 -0.01592901535332203,
 -0.007923311553895473,
 0.00048190419329330325,
 -0.037680357694625854,
 0.01792014203965664,
 -0.003410662990063429,
 -0.022053446620702744,
 -0.027189180254936218,
 8.732635615160689e-05,
 0.010559838265180588,
 0.03427484631538391,
 -0.046990592032670975,
 -0.005496196448802948,
 -0.01728847436606884,
 -0.02057039923965931,
 0.0010513493325561285,
 0.00864423718303442,
 0.013354281894862652,
 -0.00864423718303442,
 -0.0016323806485161185,
 -0.00306908180937171,
 -0.018057459965348244,
 -0.016189921647310257,
 -0.004720343742519617,
 0.0036698526237159967,
 -0.01495404914021492,
 -0.0013611754402518272,
 -0.027793383225798607,
 -0.00828720722347498,
 -0.00948874931782484,
 -0.003513652365654707,
 0.005266187246888876,
 0.018373293802142143,
 -0.010607900097966194,
 0.014720606617629528,
 0.042376670986413956,
 -0.0015645794

## c) Load parsed job info to Opensearch 

In [4]:
import boto3
import requests
from requests_aws4auth import AWS4Auth
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

region = "us-east-2"  # e.g. us-west-1
service = "es"
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(
    os.getenv("AWS_ACCESS_KEY_ID"),
    os.getenv("AWS_SECRET_ACCESS_KEY"),
    region,
    service,
    session_token=credentials.token,
)

# The OpenSearch domain endpoint
host = "search-ui-test-j4slt3fz7uu5ahsm6vgu3hwqem.us-east-2.es.amazonaws.com"  # e.g. https://search-mydomain.us-west-1.es.amazonaws.com

# # The index and document type
# index = "jb-index"
# type = "_doc"

# # Initialize OpenSearch client
# es = Elasticsearch(
#     hosts=[{"host": host, "port": 443}],
#     http_auth=awsauth,
#     # use_ssl=True,
#     verify_certs=True,
#     node_class="requests"
#     # connection_class=RequestsHttpConnection,
# )

# # JSON document you want to save
# document = {"id": "1", "title": "Example title", "content": "Sample content"}

# # Save the document in OpenSearch
# response = es.index(index=index_name, id=document["id"], body=document)
# print(response)

In [5]:
client = boto3.client("opensearch")
response = client.describe_domains(
    DomainNames=[
        "vs-test-domain",
    ]
)
print(response)

{'ResponseMetadata': {'RequestId': '9f58be78-0a3c-4a33-8470-f2eeb7e1a0a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9f58be78-0a3c-4a33-8470-f2eeb7e1a0a4', 'content-type': 'application/json', 'content-length': '23', 'date': 'Sat, 27 Jan 2024 20:52:53 GMT'}, 'RetryAttempts': 0}, 'DomainStatusList': []}


In [6]:
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [18]:
index_name = "jobs-index-vector"

client.indices.exists(index=index_name)

True

In [21]:
def create_index(index_name):
    # Create an index with non-default settings.
    # https://docs.aws.amazon.com/opensearch-service/latest/developerguide/sizing-domains.html#bp-sharding
    # shards should correspond to 10-30GB where search latency is objective
    # should be 30-50GB for write-heavy jobs like log analytics
    # by default, each index == 5 primary shards + 1 replica/primary == 10x total shards
    # replica shard == copy of a primary shard
    # shards are distributed amongst nodes for resilience
    # index > shards > multiple nodes (assuming you have more than 1)
    # lower shard count == faster reads
    # higher shard count == faster writes
    index_body = {"settings": {"index": {"number_of_shards": 1}}}

    try: 
        if client.indices.exists(index=index_name):
            print(f"Index {index_name} already exists")
            return
        response = client.indices.create(index_name, body=index_body)
        print("\nCreating index:")
        print(response)
    except Exception as e:
        print(e)


create_index(index_name)
print()

AuthorizationException(403, '{"message":"The security token included in the request is invalid."}')



In [6]:
# Add a document to the index.
document = {"title": "Moneyball", "director": "Bennett Miller", "year": "2011"}
id = "1"

response = client.index(index=index_name, body=document, id=id, refresh=True)

print("\nAdding document:")
print(response)


Adding document:
{'_index': 'python-test-index-2', '_id': '1', '_version': 2, 'result': 'updated', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [12]:
from llm_chain.chain.parser_resume import ResumeParserLLM, json_sample_schema_resume
from llm_chain.embedding import EmbeddingModel


embedder = EmbeddingModel(model="text-embedding-ada-002")
chat_resume = ResumeParserLLM(model="gpt-3.5-turbo-1106")

In [ ]:
sample_resume = "Yusa Li\nToronto, ON, M4Y 0E8 | 780-707-7844 | yusa.li@hotmail.com | linkedin.com/in/yusa-li/ | github.com/yussaaa\nCurious sponge that always absorbs trending technologies and aims to tackle impactful problems\n\nSKILLS\nCloud & Infrastructure: Azure · AWS · GCP · Spark · Kubeflow · Databricks · Terraform · Docker · Kubernetes\nMachine Learning:\nPyTorch · TensorFlow · scikit-learn · Pandas · NumPy · Spacy · Label Studio · Great Expectations · MLflow\nProgramming:\nPython · Rust · Golang · SQL · NoSQL · Shell · Git · Flask · LangChain · Airflow · Kafka · RabbitMQ\nAnalytics & Visualization: Tableau · Power BI · SparkSQL · BigQuery · Azure Synapse · Matplotlib · Seaborn · Plotly\nCertification:\nAzure Data Scientist Associate · AWS Solution Architect Associate · GCP Professional MLE (Exam scheduled)\n\nPROFESSIONAL EXPERIENCE\nStatistic Canada – Consumer Price Division\nToronto\nMachine Learning Engineer\nFeb 2022 – June 2023\n• Developed and deployed MLOps data validation pipeline on AzureML. Built data validation rules using Great Expectations\n• Contributed to Great Expectations open-source community with custom expectation for null value check\n• Orchestrated pipeline components such as classifier, data validation, outlier detection resulting 90% reduction of work effort\n• Provisioned infrastructure such as cluster size, compute size and Azure ML workspace using Terraform\n• Built and deploy CI/CD pipeline on GitLab with unit testing with pytest and linting checks\n• Fine-tuned BERT based model with PyTorch on NLP classification and NER task. Tracked experiments using Weight and Bias\nData Scientist\nDec 2020 – Feb 2022\n• Developed a Named Entity Recognition model for extracting electronic product features. Full ML cycle was practiced from\ndata labelling (hosting Label Studio docker image on Azure App Service), concept validation to model building (Spacy)\n• Scraped text description data using Beautiful Soup and selenium. Prototyped NER service with Azure cognitive service\n• Trained and deployed NLP classification model on Azure ML, experimented and tracked with MLflow\n• Built Amazon product historical price tracking tool using Keepa and Python. Visualized in PowerBI dashboard\nBMO-Global Asset Management\nToronto\nDatabase Analyst\nMar 2020 – May 2020\n• Automated portfolio reporting process by extracting and transforming millions of rows of equity data using Azure Synapse\n• Designed and executed testing plan to evaluate and ensure data quality using SQL, Excel functions, pivot tables, Power BI\n• Communicated with stakeholders to gather accurate requirements for the Business Intelligence report\n\nPROJECTS\nJob recommender system with LLM, RAG and LangChain (Ongoing)\n• Built a job recommender app to 1. Extract and summarize PDF resumes 2. Parse LinkedIn job postings 3. Periodically parse\nlatest job postings Apify 4. Send user daily recommended job email with unmatched skill summary\n• Implemented job parsing with Beautiful Soup and Selenium; Front-end build with Streamlit; Emails sent with AWS SNS\n• Automated a serverless ETL pipeline using AWS Lambda and EventBridge, storing cleaned job descriptions as JSON in\nOpenSearch and saving transformed embeddings to Pinecone VectorDB\n• Engineered the OpenAI prompts to conduct summarization and comparison tasks; Connected Chains using LangChain\n• Recommend top matched 20 jobs based on semantic search result between embeddings of user’s resume and JDs\nFull Stack Transfer learning food classifier with PyTorch and TensorFlow\n• Loaded pre-trained feature and fine-tuned the model using the Food101 dataset\n• Compared between 30Mb EffnetB2 and 295Mb ViT-b-16 model. Opted EffnetB2 for production due to its size advantage\n• Built Gradio app with the trained model and deployed the app to Hugging Face Space\n\nEDUCATION\nMaster of Engineering, Industrial Engineering (Emphasis on Analytics / Data Science)\nUniversity of Toronto\n\n2018 - 2019\n\nBachelor of Science, Mechanical Engineering\nUniversity of Alberta\n\n2014 - 2018\n\n\x0c"

parsed_resume = chat_resume.parse(
    json_input=sample_resume, json_sample_schema=json_sample_schema_resume
)

serialized_string = ", ".join(f"{k}: {v}" for k, v in parsed_resume.items() if v)

In [13]:
embedding_resume = embedder.get_embedding(serialized_string)

In [20]:
# Search for the document.
q = embedding_resume
# query = {
#     "size": 5,
#     "query": {"multi_match": {"query": q, "fields": ["embedding"]}},
# }

query = {"size": 20, "query": {"knn": {"embedding": {"vector": q, "k": 10}}}}

response = client.search(body=query, index=index_name)
print("\nSearch results:")
print(response)


Search results:
{'took': 369, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 78, 'relation': 'eq'}, 'max_score': 0.77768666, 'hits': [{'_index': 'jobs-index-vector', '_id': 'kuNTPo0BnGlYq3t-c7mP', '_score': 0.77768666, '_source': {'skills': {'databases': ['SQL', 'NoSQL'], 'deep_learning_concepts': [], 'cloud_platforms': ['AWS'], 'large_models': [], 'machine_learning_frameworks': ['AWS Sagemaker'], 'web_dev_back_end': [], 'big_data_technologies': ['Apache Spark', 'AWS Glue', 'Hadoop'], 'web_dev_front_end': [], 'devops_tools': ['AWS CloudFormation', 'CI/CD'], 'data_visualization_tools': [], 'machine_learning_operations': [], 'statistical_analysis_methods': [], 'programming_languages': ['Python'], 'data_engineering_tools': [], 'natural_language_processing': []}, 'company_culture': ['mission-driven', 'highly collaborative'], 'job_benefits': {'retirement_plan_matching': False, 'remote_work_policy': {'partial_remote_optio

In [25]:
len(response["hits"]['hits'])

20

In [9]:
import pprint
pprint.pprint(response)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'eeOvOY0BnGlYq3t-F7nF',
                    '_index': 'jobs-index',
                    '_score': 2.8503165,
                    '_source': {'company_culture': ['work-life balance',
                                                    'team environment',
                                                    'flexible hours',
                                                    'diversity'],
                                'embedding': [-0.014508680440485477,
                                              -0.018874594941735268,
                                              0.006538889370858669,
                                              -0.032318420708179474,
                                              0.0006464016623795033,
                                              0.02008587121963501,
                                              -0.03380921855568886,
                                  

## ~~Pinecone vector DB~~

### Not selected due to the free tier 1 index limitation

In [42]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")

In [40]:
import pinecone

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

NameError: name 'PINECONE_API_KEY' is not defined

In [21]:
PINECONE_INDEX = "jd-index"

In [22]:
pinecone.create_index(PINECONE_INDEX, dimension=1536, metric="cosine")
pinecone.describe_index(PINECONE_INDEX)

IndexDescription(name='jd-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [23]:
index = pinecone.Index(PINECONE_INDEX)

In [45]:
index.upsert(
    vectors=[
        {"id": "job_0", "values": job_0_emb},
    ],
    namespace="ns1",
)

{'upserted_count': 1}

In [29]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 1e-05,
 'namespaces': {'ns1': {'vector_count': 1}},
 'total_vector_count': 1}

In [ ]:
index.query(
    namespace="ns1",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=3,
    include_values=True,
)

In [ ]:
pinecone.delete_index("quickstart")

# 3. PDF Parser

In [3]:
import pprint

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Resume_Yusa_Li_MLE.pdf")
pages = loader.load_and_split()

In [5]:
pprint.pprint(pages[0])

Document(page_content='Yusa Li  \nToronto, ON, M4Y 0 E8 | 780 -707-7844 | yusa.li@hotmail.com  | linkedin.com/in/yusa -li/ | github.com/yussaaa  \n Curious sponge that always  absorb s trending technologies  and aims  to tackle  impactful  problems  \nSKILLS           \nCloud  & Infrastructure :     Azure · AWS · GCP  · Spark · Kubeflow  · Databricks · Terra form · Docker · Kubernetes  \nMachine Learning :             PyTorch  · TensorFlow  · scikit -learn  · Pandas · NumPy ·  Spacy  · Label Studio · Great Expectations  · MLflow \nProgramming :                     Python · Rust  · Golang · SQL  · NoSQL  · Shell  · Git · Flask  · LangChain  · Airflow  · Kafka · RabbitMQ  \nAnalytics & Visualization : Tableau · Power BI · SparkSQL  · BigQuery · Azure Synapse  · Matplotlib · Seaborn · Plotly  \nCertification :                       Azure Data Scientist Associate  · AWS Solution Arch itect Associate · GCP Professional MLE  (Exam scheduled)  \nPROFESSIONAL  EXPERIENCE   \nStatistic Canada –

In [9]:
whole_resume = ""
for page in pages:
    whole_resume += "\n\n" + page.page_content

In [10]:
whole_resume

'\n\nYusa Li  \nToronto, ON, M4Y 0 E8 | 780 -707-7844 | yusa.li@hotmail.com  | linkedin.com/in/yusa -li/ | github.com/yussaaa  \n Curious sponge that always  absorb s trending technologies  and aims  to tackle  impactful  problems  \nSKILLS           \nCloud  & Infrastructure :     Azure · AWS · GCP  · Spark · Kubeflow  · Databricks · Terra form · Docker · Kubernetes  \nMachine Learning :             PyTorch  · TensorFlow  · scikit -learn  · Pandas · NumPy ·  Spacy  · Label Studio · Great Expectations  · MLflow \nProgramming :                     Python · Rust  · Golang · SQL  · NoSQL  · Shell  · Git · Flask  · LangChain  · Airflow  · Kafka · RabbitMQ  \nAnalytics & Visualization : Tableau · Power BI · SparkSQL  · BigQuery · Azure Synapse  · Matplotlib · Seaborn · Plotly  \nCertification :                       Azure Data Scientist Associate  · AWS Solution Arch itect Associate · GCP Professional MLE  (Exam scheduled)  \nPROFESSIONAL  EXPERIENCE   \nStatistic Canada – Consumer Price Di

In [6]:
pages[0].page_content

'Yusa Li  \nToronto, ON, M4Y 0 E8 | 780 -707-7844 | yusa.li@hotmail.com  | linkedin.com/in/yusa -li/ | github.com/yussaaa  \n Curious sponge that always  absorb s trending technologies  and aims  to tackle  impactful  problems  \nSKILLS           \nCloud  & Infrastructure :     Azure · AWS · GCP  · Spark · Kubeflow  · Databricks · Terra form · Docker · Kubernetes  \nMachine Learning :             PyTorch  · TensorFlow  · scikit -learn  · Pandas · NumPy ·  Spacy  · Label Studio · Great Expectations  · MLflow \nProgramming :                     Python · Rust  · Golang · SQL  · NoSQL  · Shell  · Git · Flask  · LangChain  · Airflow  · Kafka · RabbitMQ  \nAnalytics & Visualization : Tableau · Power BI · SparkSQL  · BigQuery · Azure Synapse  · Matplotlib · Seaborn · Plotly  \nCertification :                       Azure Data Scientist Associate  · AWS Solution Arch itect Associate · GCP Professional MLE  (Exam scheduled)  \nPROFESSIONAL  EXPERIENCE   \nStatistic Canada – Consumer Price Divisi

In [7]:
import pdftotext

# Load your PDF
pdf_path = "../data/Resume_Yusa_Li_MLE.pdf"
with open(pdf_path, "rb") as f:
    pdf = pdftotext.PDF(f)

# Read all the text into one string
pdftotext_text = "\n\n".join(pdf)

In [8]:
pdftotext_text

'Yusa Li\nToronto, ON, M4Y 0E8 | 780-707-7844 | yusa.li@hotmail.com | linkedin.com/in/yusa-li/ | github.com/yussaaa\nCurious sponge that always absorbs trending technologies and aims to tackle impactful problems\n\nSKILLS\nCloud & Infrastructure: Azure · AWS · GCP · Spark · Kubeflow · Databricks · Terraform · Docker · Kubernetes\nMachine Learning:\nPyTorch · TensorFlow · scikit-learn · Pandas · NumPy · Spacy · Label Studio · Great Expectations · MLflow\nProgramming:\nPython · Rust · Golang · SQL · NoSQL · Shell · Git · Flask · LangChain · Airflow · Kafka · RabbitMQ\nAnalytics & Visualization: Tableau · Power BI · SparkSQL · BigQuery · Azure Synapse · Matplotlib · Seaborn · Plotly\nCertification:\nAzure Data Scientist Associate · AWS Solution Architect Associate · GCP Professional MLE (Exam scheduled)\n\nPROFESSIONAL EXPERIENCE\nStatistic Canada – Consumer Price Division\nToronto\nMachine Learning Engineer\nFeb 2022 – June 2023\n• Developed and deployed MLOps data validation pipeline on